## Accessing NCBI databases with Biopython


we will look at how to access such databases at the National Center for Biotechnology Information (NCBI). We will not only discuss GenBank, but also other databases at NCBI. Many people refer (wrongly) to the whole set of NCBI databases as GenBank, but NCBI includes the nucleotide database and many others, for example, PubMed.

#### Check available databases at NCBI


Biopython provides an interface to Entrez, the data retrieval system made available by NCBI. Entrez can also be used through web browser: https://www.ncbi.nlm.nih.gov/search/

##### TIPS:
- specify an email address with your query
- avoid large number of requests (100 or more) during peak times (between 9.00 a.m. and 5.00 p.m. American Eastern Time on weekdays)
- do not post more than three queries per second (Biopython will take care of this for you)

It's not only good citizenship, but you risk getting blocked if you over use NCBI's servers (a good reason to give a real email address, because NCBI may try to contact you).

In [2]:
from Bio import Entrez, SeqIO

In [3]:
Entrez.email = "shubhamvt121@gmail.com"

**EInfo**: obtain a list of all database names accessible through Entrez



In [7]:
# This gives you the list of available databases
handle = Entrez.einfo()
rec = Entrez.read(handle)
handle.close()
print(rec.keys())

dict_keys(['DbList'])


In [8]:
rec['DbList']

['pubmed', 'protein', 'nuccore', 'ipg', 'nucleotide', 'structure', 'genome', 'annotinfo', 'assembly', 'bioproject', 'biosample', 'blastdbinfo', 'books', 'cdd', 'clinvar', 'gap', 'gapplus', 'grasp', 'dbvar', 'gene', 'gds', 'geoprofiles', 'medgen', 'mesh', 'nlmcatalog', 'omim', 'orgtrack', 'pmc', 'popset', 'proteinclusters', 'pcassay', 'protfam', 'pccompound', 'pcsubstance', 'seqannot', 'snp', 'sra', 'taxonomy', 'biocollections', 'gtr']

We will now try to find the **chloroquine resistance transporter (CRT)** gene (KM288867) in Plasmodium falciparum (the parasite that causes the deadliest form of malaria) on the nucleotide database:
**
ESear**ch: Searching the Entrez databases

Note that the standard search will limit the number of record references to 20, so if we have more, we can override retmax to desired amount of records.

In [10]:
handle = Entrez.esearch(db="nucleotide", term='CRT[Gene Name] AND "Plasmodium falciparum"[Organism]', retmax="40")
rec_list = Entrez.read(handle)
handle.close()
rec_list['Count']

'3081'

In [11]:
len(rec_list['IdList'])

40

In [12]:
rec_list['IdList']

['2587918588', '2507817686', '2507817684', '2507817682', '2507817680', '2507817678', '2507817676', '2507817674', '2507817672', '2507817670', '2507817668', '2507817666', '2507817664', '2507817662', '2507817660', '2507817658', '2507817656', '2507817654', '2507817652', '2507817650', '2507817648', '2507817646', '2507817644', '2507817642', '2507817640', '2507817638', '2507817636', '2507817634', '2507817632', '2507817630', '2507817628', '2507817626', '2507817624', '2507817622', '2507817620', '2507817618', '2507817616', '2507817614', '2507817612', '2507817610']

We now have the IDs of all of the records, but we still need to retrieve the records properly.


EFetch: Downloading full records from Entrez

Requesting a specific file format from Entrez using Bio.Entrez.efetch() requires specifying the rettype and/or retmode optional arguments. The different combinations are described for each database type on the pages linked to on NCBI efetch webpage - https://www.ncbi.nlm.nih.gov/books/NBK25499/#chapter4.EFetch

rettype - return type, gb == GenBank retmax - Total number of records from the input set to be retrieved, up to a maximum of 10,000

In [15]:
id_list = rec_list['IdList']
handle = Entrez.efetch(db='nucleotide', id=id_list, rettype='gb')

recs = list(SeqIO.parse(handle, 'gb'))
handle.close

<function TextIOWrapper.close()>

Note that we have converted an iterator (the result of SeqIO.parse) to a list. The advantage of doing this is that we can use the result as many times as we want (for example, iterate many times over), without repeating the query on the server.



In [18]:
len(recs)

40

In [20]:
recs[:10]

[SeqRecord(seq=Seq('GGTGGAGGTTCTTGTCTTGGTAAATGTGCTCATGTGTTTAAACTTATTTTTAAA...AAA'), id='OR483864.1', name='OR483864', description='Plasmodium falciparum isolate PE-26 chloroquine resistance transporter (crt) gene, partial cds', dbxrefs=[]),
 SeqRecord(seq=Seq('TGTGCTCATGTGTTTAAACTTATTTTTAAAGAGATTAAGGATAATATTTTTATT...TTG'), id='OQ672451.1', name='OQ672451', description='Plasmodium falciparum isolate ML_14 chloroquine resistance transporter (crt) gene, partial cds', dbxrefs=[]),
 SeqRecord(seq=Seq('TGTGCTCATGTGTTTAAACTTATTTTTAAAGAGATTAAGGATAATATTTTTATT...TTG'), id='OQ672450.1', name='OQ672450', description='Plasmodium falciparum isolate ML_13 chloroquine resistance transporter (crt) gene, partial cds', dbxrefs=[]),
 SeqRecord(seq=Seq('TGTGCTCATGTGTTTAAACTTATTTTTAAAGAGATTAAGGATAATATTTTTATT...TTG'), id='OQ672449.1', name='OQ672449', description='Plasmodium falciparum isolate ML_12 chloroquine resistance transporter (crt) gene, partial cds', dbxrefs=[]),
 SeqRecord(seq=Seq('TGTGCTCATGTGTTTA

In [21]:
for rec in recs:
    if rec.name == 'KM288867': #Try finding CRT gene in 40 records we fetched
        break
print(rec.name)
print(rec.description)

OQ672413
Plasmodium falciparum isolate MAO_27 chloroquine resistance transporter (crt) gene, partial cds


In [22]:
str(rec.seq)

'TGTGCTCATGTGTTTAAACTTATTTTTAAAGAGATTAAGGATAATATTTTTATTTATATTTTAAGTATTATTTATTTAAGTGTATCTGTAATGAATACAATTTTTGCTAAAAGAACTTTAAACAAAATTGGTAACTATAGTTTTG'